**Finding motifs with the highest frequency in the string?"**

Our aim is to turn the biological challenge of finding regulatory motifs into a computational problem. Below, we have implanted a 15-mer hidden message at a randomly selected position in each of ten randomly generated DNA strings. This example mimics a transcription factor binding site hiding in the upstream regions of ten genes.**

1 "atgaccgggatactgataaaaaaaagggggggggcgtacacattagataaacgtatgaagtacgttagactcggcgccgccg"
2 "acccctattttttgagcagatttagtgacctggaaaaaaaatttgagtacaaaacttttccgaataaaaaaaaaggggggga"
3 "tgagtatccctgggatgacttaaaaaaaagggggggtgctctcccgatttttgaatatgtaggatcattcgccagggtccga"
4 "gctgagaattggatgaaaaaaaagggggggtccacgcaatcgcgaaccaacgcggacccaaaggcaagaccgataaaggaga"
5 "tcccttttgcggtaatgtgccgggaggctggttacgtagggaagccctaacggacttaataaaaaaaagggggggcttatag"
6 "gtcaatcatgttcttgtgaatggatttaaaaaaaaggggggggaccgcttggcgcacccaaattcagtgtgggcgagcgcaa"
7 "cggttttggcccttgttagaggcccccgtaaaaaaaagggggggcaattatgagagagctaatctatcgcgtgcgtgttcat"
8 "aacttgagttaaaaaaaagggggggctggggcacatacaagaggagtcttccttatcagttaatgctgtatgacactatgta"
9 "ttggcccattggctaaaagcccaacttgacaaatggaagatagaatccttgcataaaaaaaagggggggaccgaaagggaag"
10 "ctggtgagcaacgacagattcttacgtgcattagctcgcttccggggatctaatagcacgaagcttaaaaaaaaggggggga"

Can you find the implanted hidden message?

In [2]:
def FrequentWords(Text, k):
    Count = {}
    for i in range(0, len(Text) - k):
        Pattern = Text[i : k + i]
        Count[Pattern] = Text.count(Pattern)

    max_freq = sorted(set(Count.values()))[-1] # sort the dictionary values and -1 picks the largest value from among the list
    
    print(f"The k-mer of length {k} with maximum frequency is:")
    for j,k in Count.items():
        if k == max_freq:
            print(j)

dna = open('data.txt','r').read()
FrequentWords(dna,15)

The k-mer of length 15 with maximum frequency is:
aaaaaaaaggggggg


**A brute force algorithm for motif finding:**

Given a collection of strings Dna and an integer d, a k-mer is a (k,d)-motif if it appears in every string from Dna with at most d mismatches. For example, the implanted 15-mer in the strings above represents a (15,4)-motif.

Implanted Motif Problem: Find all (k, d)-motifs in a collection of strings.

Input: A collection of strings Dna, and integers k and d.
Output: All (k, d)-motifs in Dna.

**Note:**
If a kmer of length k generated from one string has its neighbour matching with kmer generated from all other strings with a mismatch d, then the neighbour is considered a motif

In [44]:
def HammingDistance(p, q):
    mismatch = 0
    for i in range(len(p)):
        if p[i] != q[i]:
            mismatch += 1
    return mismatch

In [9]:
def Neighbors(Pattern, d):
    if d == 0:
        return {Pattern}
    if len(Pattern) == 1:
        return {'A', 'C', 'G', 'T'}

    neighborhood = set()
    suffix_neighbors = Neighbors(Pattern[1:], d)
    for neighbor in suffix_neighbors:
        if HammingDistance(Pattern[1:], neighbor) < d:
            for nucleotide in {'A', 'C', 'G', 'T'}:
                neighborhood.add(nucleotide + neighbor)
        else:
            neighborhood.add(Pattern[0] + neighbor)
    return neighborhood

In [42]:
def MotifEnumeration(Dna, k, d):
    Patterns = []
    potential_motif = []
    strings = Dna.split()
    for i in strings:
        for j in range(len(i) - k + 1):
            k_mer = i[j:j+k]
            neighbourhood = Neighbors(k_mer, d)
            for neighbour in neighbourhood:
                for m in strings:
                    if m != i:
                        for l in range(len(m) - k + 1):
                            mismatch = HammingDistance(neighbour, m[l:l+k])
                            if mismatch <= d:
                                potential_motif.append(m)
                if len(set(potential_motif)) == len(strings) - 1:
                    Patterns.append(neighbour)
                potential_motif = []
    return set(Patterns)

MotifEnumeration('GCATCCAAGAAGACTGGTAAATGCA GAAGGTCGGGTACGTGTGCATGTGC GCGGCGAGAGTAATAAAAGCTTACA AGACAGGTGAGGCACTTGTTCAAGC CGAGCTAAGAGGGCGAGACCGATCT TTCATTTTGCTAAGTGGTTCGGTAC', 5, 2)

{'AAAAG',
 'AAACG',
 'AAACT',
 'AAAGA',
 'AAAGC',
 'AAAGG',
 'AAAGT',
 'AAATC',
 'AAATG',
 'AACAT',
 'AACGG',
 'AACGT',
 'AACTC',
 'AACTG',
 'AACTT',
 'AAGAA',
 'AAGAC',
 'AAGAG',
 'AAGAT',
 'AAGCA',
 'AAGCC',
 'AAGCG',
 'AAGCT',
 'AAGGA',
 'AAGGC',
 'AAGGG',
 'AAGGT',
 'AAGTA',
 'AAGTC',
 'AAGTG',
 'AAGTT',
 'AATCG',
 'AATGA',
 'AATGC',
 'AATGG',
 'AATGT',
 'AATTG',
 'AATTT',
 'ACAAG',
 'ACAGG',
 'ACAGT',
 'ACATA',
 'ACATC',
 'ACATG',
 'ACCGT',
 'ACCTA',
 'ACCTG',
 'ACGAA',
 'ACGAG',
 'ACGCG',
 'ACGCT',
 'ACGGA',
 'ACGGC',
 'ACGGG',
 'ACGGT',
 'ACGTA',
 'ACGTC',
 'ACGTG',
 'ACTAC',
 'ACTAG',
 'ACTGC',
 'ACTGT',
 'ACTTA',
 'AGAAC',
 'AGACG',
 'AGAGC',
 'AGAGG',
 'AGAGT',
 'AGATA',
 'AGATC',
 'AGATG',
 'AGATT',
 'AGCAA',
 'AGCAC',
 'AGCAG',
 'AGCAT',
 'AGCCG',
 'AGCGC',
 'AGCGG',
 'AGCTA',
 'AGCTG',
 'AGCTT',
 'AGGAA',
 'AGGAC',
 'AGGAG',
 'AGGAT',
 'AGGCA',
 'AGGCG',
 'AGGCT',
 'AGGGA',
 'AGGGC',
 'AGGGG',
 'AGGGT',
 'AGGTA',
 'AGGTC',
 'AGGTG',
 'AGGTT',
 'AGTAA',
 'AGTAC',
 'AGTAG',


**Compute the entropy of the NF-kB matrix**

Entropy = -p . log 2 (p)

Entropy is an indication of conservation calculated from the entropy of each column of the matrix. The more conserved the column, the smaller its entropy.

Entropy is a better metric to score motifs

In [15]:
import math

def entropy(strings):
    dna = strings.split()
    num_dna = len(dna)
    entropy_cal = 0
    a, t, g, c = 0, 0, 0, 0
    for i in range(0,len(dna[0])):
        for j in dna:
            if j[i] == 'A':
                a += 1
            elif j[i] == 'T':
                t += 1
            elif j[i] == 'G':
                g += 1
            else:
                c += 1
                
        a /= num_dna
        t /= num_dna
        g /= num_dna
        c /= num_dna
        if a > 0:
            entropy_cal += -(a * math.log2(a))
        if t > 0:
            entropy_cal += -(t * math.log2(t))
        if g > 0:
            entropy_cal += -(g * math.log2(g))
        if c > 0:
            entropy_cal += -(c * math.log2(c))
        a, t, g, c = 0, 0, 0, 0
    
    return entropy_cal
    
entropy("TCGGGGGTTTTT CCGGTGACTTAC ACGGGGATTTTC TTGGGGACTTTT AAGGGGACTTCC TTGGGGACTTCC TCGGGGATTCAT TCGGGGATTCCT TAGGGGAACTAC TCGGGTATAACC")

9.916290005356972

In [52]:
def DistanceBetweenPatternAndStrings(Pattern, Dna):
    k = len(Pattern)
    distance = sum(min(HammingDistance(Pattern, text[i:i+k]) for i in range(len(text) - k + 1)) for text in Dna.split())
    return distance

DistanceBetweenPatternAndStrings("ATGATTA", "CCACGTCGCGAACCATACCCGCCATCGTTCGTGCGGTTCCTAGTACAGGGACACTGACGAAACTAATATAGGAGCGAGTAGTCTTGATATTCCATTGACCAAT AGATTTGGTTGTTTCGTGGTGTCGAATTGGTCGCAGATGATTCGCCTGGTAGGCGATTTCACCCTTGCCCCGTTTAGGGAAATACTAGCCACTACACTGATCT TAGACCAATATTGTAAATTGCGCTGCGGCAGTGCGATTATCGGCGATGCAAGCAAGGGGGCAACTCTCATGAGCATATTTGCTTCGCAAGCCTCCAGCAGTAT GGGGCCCAGCTGATGGCTGTGGGTGTGTGGCATTAAACAGTATGTACTTAATGACAGGTGCGTAAACGCATCATAGGTTTACGTTACGACGTTCTCTGCGAGA GATGTTAGCATATTCCCACCAGCCAGACCACAGAATACGGAGACCACGGTGCTCCCTCCATGAACCCGAGTGCTAGGTGTACGACAACAAAACACTTAAGGCA CCTAGAGCGTTAGAAGAGCAAGCGAAATTTTAAAAAATACGCTTTGTCAACAAACTGCGCAACAGCATATGGCTGTGCGGGAGTAGGAAGTGATATGCGTTTC CAATCATATCCAGATAGTGGTATCAGCAGTGGTGCAATGGGCAGAGTCAGGCATAGATCGGCGCGTAAATACTTATCGTGCCTAGGTTGACGATCCATAGCTA GACTCCACAATGAACAGCGAGCTTGGGTCTATTTGGCGGTCTAACCTTAATGTTTTCTGAACGCTTTTTTTACTCTTCACAGAGCATACAAACCACCGATAGA TACCATTCGAGATGAATGTCAGCCAAAATCCCGTCTGTCCAAAGTCCCGTGACTTATCGCGCAGAATTCAAGCTGTTTATGACCGACGATACAGATCATCTGA GATGATTTATTGGCGATCCTAGTGAATGTGTAGTAGGGGCAGCTTTAGGGAAGTCAAATTCCTAGAGTTCGAAGTTCTGTCTACAAAAATCAGATGTCGCTCG TCAGTTACGTACCGCAGGTCAGCTGAGATGTAATAGGCTCCACCTGTTACGGGCACTCTGGATCGGTGTAAGAATTGGGTACACAGGTAACGGCGAGTGTAGA TCGAGAACCCAGACTGACTTGAGCTGACGGCTTCGGTACAGATGAATTACGGTATCACGTCTATGCTGTCGCCATTAGGCTCAAACCGTAAAGAAGACATGTG TCGTGGTATCATCAAAGACTAATTCTCGACAGGACAAACGCGACTGGCGGATAATAGGATACGATTGGCGATGTGCCTACGTGGATTCGGCGGGGTCAGGCAT GTCGGGATCCTTATCTTGGCATTTCATGCCCCAGAATGCATAGCCATTCACCTTAGTAAAAATGCCCTTACGTTGCACTCTTTCCAGGCGCACCGCTGCGCTC GAGCAGTTGTAAGCCAATTACTTGGGAGAGTAGGCTAGTGATAGCCGCACATGCAGAACCGATTTGGCCGGTAACCTATTACCAGACATCTGACTCGTATGAA GACCGCCCCAAGTCCGTGGACTTTAGCTCGGATCCGTACACCTTAGCGTCACTTCACGCCCTTCCGACCGCGAGGGTAGATTACCTGGCGCCTAGATAATCAC TAACATCCCCGCGTGTTGGGACGATGTTCCATTTATGGGAAAGGCACAAGAAATCTGAAAGTAACTAGGGTAATAGATCGTCGTGTCATACAATTTTTAACGT TGCCAGGGTAACTAGAGCACCGTCTAGGCAACGTCAAACGTGACAGGTGATGCGCCGCTTCTACGAAGTGTCTAGCCGACACTTCGGAATAACGACTGCACTG GGATTTAACGTACAGGAAACCGTCTCTGCTGCCAAGGCCCCACAAACTACTCAGAGAGAAAGTGTCGTGGCCAGCAGAGAGGCGTCGGTTGCATGCGAATCGG GGAATGTAACATAATGCCGTACCCCCAGGGATTATTTGTGAAGCATTTCTACTCACGAATCGGCCTTTAGCCCGAAGCAACGTATGGTCATACTTTGCTAAAA TTTAAGTCACTCGTTACCTATGACTTCTGGTCGTATATCTATATTACCTGTCGATGAAGATTGCAAGAGTGGGCAATTCTTACGTTCTCAATGGAGATATATT GTACTAGTCACTGCTGGCATATTCCAGAGGTTTGGGTTTAAACAGTCCAGCGGGGGTCTCCACTTGCCCGTAAAACCGCCCAATCGATGTCATTTTGGGATTG TGGTTCTTTCGGCTCGGAATCCTATCTTTATAAAAATAATGGCAGACCGTTTACGTTGCTAGCTGTTTTCTCCACTCAGAAATGTGGACAGCAGACGGGAGTT TTGGGTTGTGGAGAGCAATGGTTCAATAGTTTCTGGACGTTGCGAATGGTAATATGTATCACGACCGTCACGAATCTTTCACGGCATGATGCGATCGTGTTAA TATTCAAGCCCTATCCCCGGAACCATTTTCCACGTGGTATCAAAGGTCTTGTGCCTCCGGACACCATATGCGGCACATGGTTTTCCAGCTCAGATTGTTCAAG CCACCTGTACCCAGAGTATGCGAAACCTGACGGTCGCCCAGGTTCGGTTTCAAGGAGACCTGGCTGACGGATTTACAAAGGCTGTTTTTCTACTATGGTACGG GGACACATTTGACGGTCTAGTCGCGCCATGCCTCACTAACGCTGTTCGAATGTTCTCAATTGATTCGATTTCACGCGAGCAGATGTCCCTGCGTAACAGTCAG CAAAGTGGCCATAAACCTGATTGAGGCACTGTCCTCCGTAGGGGGGTCTATTGATCTGTCCGACCGTAAAGACAAGAAAGTCCTGTGTGAGACAACCGACAAA TGTTAATACCGAATGCCTACGGCGGCTCGGAGGACGAAATTAAGACCTCTGGTTGCCTCGAGATTGTGTTGTCTACCGCGGCTCAATACAGGCAAATCCCGGT GACTACTCTCATGTGGTAACCTTCATTATAGGTGCTACTGGTTGACGGGTGAGATGACGAATTGATAACGGCAGTGTCTGACACAGTAGTATTGAACAACGTC")

67

In [58]:
def MedianString(k, Dna):
    alphabet = ['A', 'C', 'G', 'T']
    distance = float('inf')
    median = ''

    for i in range(4 ** k):
        pattern = ''
        index = i
        for _ in range(k):
            pattern = alphabet[index % 4] + pattern
            '''
            Since there are four possible nucleotides (A, C, G, T), using remainder division by 4 ensures that the 
            remainder will be in the range [0, 1, 2, 3]
            '''
            index //= 4
            '''
            After extracting the nucleotide, index needs to be updated to consider the next bits for the next iteration
            '''
            
        if distance > DistanceBetweenPatternAndStrings(pattern, Dna):
            distance = DistanceBetweenPatternAndStrings(pattern, Dna)
            median = pattern

    return median

MedianString(6, "TGCCGGAATAGTCCTCGTTCCTGAAGTGCTAGCTAGAAGCTA CACGTAGCTATGCTGTAGCAGGACCGCCAAAGCGCTTCGTGC TTGGAGGTCTGTTAATTCAGGGCTGACCACGTCTACCGTAGG CAGATTACGTAAAGGGCTATCCTGTAGGACACCGGTCCATAA ACGTCTATCTGCAGGGAGTTTTCCAGCGCTGTCAGGGGGGCG CGGCTGTGTGTAGGTAGCAACGGTTGGATTACATAGAGCGCT ATGCTGAGGGCTTGCAATACAGGCACATCAACGATTGGTGTT AGCGTAGACGCGAGAGCTCACCAAAGCACGATCCGCCTAACC GTCCCGGTAATTTACACCCCCGTAAGTGCTCCGTATGAGAAC GTTGCACTGCACGCAACACCTCCTGAAATCTATCGGAGGGCT")

'AGGGCT'

In [59]:
def median_string(k, dna):
    distance = float('inf')
    pattern = ""

    for i in range(len(dna[0]) - k + 1):
        current_pattern = dna[0][i:i+k]
        current_distance = sum(min(HammingDistance(current_pattern, seq[i:i+k]) for seq in dna) for i in range(len(dna[0]) - k + 1))

        if current_distance < distance:
            distance = current_distance
            pattern = current_pattern

    return pattern

median_string(6, ["AAGCTTTCCTTTACAGTTTTCCAGCACAGCCGGAGGTGAATG", "GAAAGGACAGTTAACGTGACTGACGCGCTCCGGAATATTAGA", "ATCGGGTAGGTTGAGGTGAAAGTTCTAAACAAAACGTAGACG", "GGACCATCACGGCCTGGATGGCGAACTGACAGAGTTGCTTTT", "TGACCGAGTTCTTAGAGGTTCCGGACAGTGAACCCCAGAGTT", "GAAGTGAGAAAGGACGTTATAGTTCCTAAATTCAGAACCATG", "ACAGTTAAGTGTCCTCGGGGTTTTGTACACTGAGTCGTTGGA", "ACATAAGTAGCGGAGTGAATGACGTAGCAGATAGTTAGGGTT", "TAAATCAATTTAGTTCCGATAGTTGGCTGTTTCGCAAAATCA", "CGCCCGCCATCAAACTCTTCGGGCTTACCGTTTGGGAAAGTT"])

'AGTTTT'

In [40]:
import pandas as pd

def Profile_MostProbable_kmer(string, k, profile):
    kmer_score = {}
    for i in range(len(string) - k + 1):
        kmer = string[i:i+k]
        score = 1
        for j in range(len(kmer)):
            score *= profile.loc[kmer[j], j]
        kmer_score[kmer] = score
    
    max_score = max(kmer_score.values())
    print(f"Most probable kmer is {list(kmer_score.keys())[list(kmer_score.values()).index(max_score)]} with score {max_score}")
    return list(kmer_score.keys())[list(kmer_score.values()).index(max_score)]
            
data = [[0.277, 0.265, 0.265, 0.229, 0.217, 0.313, 0.229, 0.313, 0.277, 0.181, 0.193, 0.229], [0.265, 0.301, 0.265, 0.253, 0.265, 0.193, 0.157, 0.253, 0.277, 0.229, 0.337, 0.217], [0.217, 0.241, 0.277, 0.325, 0.289, 0.229, 0.313, 0.181, 0.205, 0.289, 0.205, 0.253], [0.241, 0.193, 0.193, 0.193, 0.229, 0.265, 0.301, 0.253, 0.241, 0.301, 0.265, 0.301]]
df = pd.DataFrame(data, index = ['A', 'C', 'G', 'T'])
Profile_MostProbable_kmer('TTGGACATAACGGTAGTGCATAACCGTAATACTTTATAACCTTCAGATCCACCACAACAGCAACCGCACTTATGCTCCCAGTCACCAATGATAGCAAGACGGCGCTCGCCTCTATGAAATGTCTCGCGAGTGCCTATTAGAGGCTCTTACAGTACACAGGTCAGACGAAAGGTCCAAGTATTCCGTGTCCCTACCAAATGACCCTGGTTTAGGCTGAAACTCAGGCCTAATAGCTCACGCTGTAACGTTAGCCTTGCCAAGGATCATAAGGAGTGCCCGCATGCCGGGTCAGCCTATACAATTGAATCTAACGGCATACCTCTAGATTTCAAAGAAGGGGACGGGTCCCGTCAAAGTAAAAGAATACGAGGGATCGTCACTATATTGCTTTAAGAGGTTCTTGGTAGGTACGAATTCTTCGACTATAAAGAGAGCCTCACCGCTAACAACCATCGAAAGAGAAACATAAAATTGATCGGCCGACAGAATGACATCCAAACAGTGCACGGATGCCAAGGGTGCTTGGGACCTCGAGCTGAAAGAGCCTGAACGTATTATCTGACAGCTGAGTCGATTCATATGAGAGACCTCAACTAAGTGTCGCTTAGATAGCAACCGAATAGGCCCGTGCAAGTGGGCTTAACTATCTGAGGAGATCTGTAGGACTTAGGAAAGCGGGCGTGGGTGCTGCTGTCTAGCCCGATGAATCGTAACGAGATCTGAGTTATGCAAGAGAATGTTGTTCAGCTCCTTTGGGGTCTCACTTCTGAATCCGGTAATTGTTTCTGACCCCCTGATATAACAAGATCTTATACATTTTTAACAGCTACCTAATCGCGAATCAGACCCCTGCTCCGGTTCGGTTTGCAACTGTGGCTTCCCAAGGGAGACGTCCGGAGATTTTGATCTGCGGAATAAGCCCAGATCCGAATAACATAGCGCACGCCTCCCTCTTACTAACCGAGTTCCATATGCACAGACGTTTTACGTGCGTTCTCAC', 12, df)

Most probable kmer is TCCGGAGATTTT with score 3.2036092294833274e-07


'TCCGGAGATTTT'

In [62]:
def profile_most_probable_kmer(text, k, profile):
    nucleotides = {'A', 'C', 'G', 'T'}
    max_probability = -1.0
    most_probable_kmer = ""
    for i in range(len(text) - k + 1):
        kmer = text[i:i+k]
        probability = 1.0
        for j, nucleotide in enumerate(kmer):
            probability *= profile[nucleotide][j]
        if probability > max_probability:
            max_probability = probability
            most_probable_kmer = kmer
    return most_probable_kmer

In [63]:
def form_profile(motifs):
    k = len(motifs[0])
    profile = {nucleotide: [0] * k for nucleotide in 'ACGT'}
    t = len(motifs)
    for i in range(k):
        column = [motif[i] for motif in motifs]
        for nucleotide in 'ACGT':
            count = column.count(nucleotide)
            profile[nucleotide][i] = count / t
    return profile

In [64]:
def score_motifs(motifs):
    k = len(motifs[0])
    t = len(motifs)
    score = 0
    for i in range(k):
        column = [motif[i] for motif in motifs]
        max_count = max(column.count(nucleotide) for nucleotide in 'ACGT')
        score += t - max_count
    return score

In [65]:
def greedy_motif_search(dna, k, t):
    best_motifs = [seq[:k] for seq in dna]
    for i in range(len(dna[0]) - k + 1):
        motif1 = dna[0][i:i+k]
        motifs = [motif1]
        for j in range(1, t):
            profile = form_profile(motifs)
            most_probable_kmer = profile_most_probable_kmer(dna[j], k, profile)
            motifs.append(most_probable_kmer)
        if score_motifs(motifs) < score_motifs(best_motifs):
            best_motifs = motifs
    return best_motifs

Dna = "GATTCAGTCTACGGATAGCAATTGCTCGATGGATTTTACGCCGCACAATTGACTATGGATGGCTCTGTTGCATTTTGATATTCTTGAGCCATCACGAACTCTTTGCGACTGGCAATTCAATCAGTCTGCCCGTGTACAAATTAGTCAGCGCGGTGA GTTTACAGACGATGAATTAATCCCCAGCTCTTACTCCCGAGTAGAGAATAAGTGCCTCCACTATGGTATACAAGTTGTTAGGTGCCTTCAATCTACTAGACCTATGATGGCTCTTCGTCAAATAAACTAACAAGGAAGGTCCTTGCAGTTACTAGA CATCTAACGTCCCATTCACTCTACTGATCGTTCAAAACTGACTGTCGAGCCACCTACTGGTGCTCTCGGCTGTTGCTAACATTACCGTGCACATGCACCTTAGCGGTCGTCGATCAGAATGATTTGTCCTCCCGTGTCAGGTGTGGTTAAGTCTGA TGGCCCCAACCTAGCAATGGAGCTAGATGTTCACCTCAATGAACCTACCCCCGCCAGCTGCTTCAAGCATAGCCTCACTGCCTACAGCCAGCTGTAAGGCTTGGCAGCTGTTCAATCAACTTACGGGTCAAAGGTAGTGCAATGTCGCGATCCTAG ATAAGTCCGTATCGTTCCAATTTTGTTACACAATTAGAAATTGGGCTGTCAAGCTCACTTTAAGAACCGACCACAAGGGCTGGCGCATATTGGTTACGAACCTCGGTGTTTAGCGCCTCCCATTCATTCTACGCAGTGGGACTAGTTCGTAGTACG CCTGGAGTACACGTTACGTGTCAAACGTATTTAGGGAAGACGCTCTGCCGTGATCTCGAAAGTTCACTCAACCAAGTTACGATAGCGTAGCTGATGCACGCACTTTCGTACTATCCAGGGCGGCGGCCTGCGCTAGTCACCTAGCTCATTGCCCAA CCCAGTGCCGGGTCGATACCGCCCCTGTTAAAGACTTCTCATAATTAAGCCTCCTTAGCAAGCCGAGATACGGACACCGGGCCCTTGCGGGCCTGCAAGTGGCACGTACCCACAGCGCAGCTAGACCTCTTCATTGTAAGAATATCTTCAATCGAC TCATCCGATCGTCACAACTGTTCATCTCCGGGCCATTTGGTTATAGTTTATTCAATCCACACCACCTCACCAAGGGAATGAAAATACCGAGACGGATGCGGAATGATGCTATGGGCAATTTCTAGCATAGTGATTGGGATGCTATATGGGAGGCGA TCACACGGTGCCTGCGTAGGGTTAGGGCAGCGGGTCTGCCCTACACAAAGGTAGACAACATGGATCTCAACCGGTCCCCACTTAGACTAGAACTCTCCGCCCGAGCCAGTTTCAATCAACGAGATTTTTCCGGACCTATTCTCGCCTCTAGTGTTA AGCTTCCCTGACCCTCGGTGGATAGTACCGTTAATACTCGGCGTCTTGTGCGACGGTATTGGTTCAGTCTACCGTTCTCGGGGCTGTACCCACCTAATTCGTGCCTGAGTCTCCCAATCAGCCTGGTTCTGGCCGACAACAGCAACTGTGCGCGCG ACACGGTTAGCTGATCAGTCTCAAAACACAGCACGTCACAAGGGCGAACAGCAGAGCGAGGGAATCGAACGCACATTGGTTTGTCTGAACCCCGCCCCTTCATTCGACTGACGAGTACGACCGAACTGTAACCTTAGGTGCTTAGGCCCATAAACA GCTTTAAATTCAGGCAGGAGGCATGTGACGATTTTAAAGCGACTATTTGCGCACACGTGCTTTAAGCAGTATTTTTCAATCCACGGTTGATGCGTCCAAGTCAGTTTCAAGACCTAGAAAATATGGATGATATTCCACCACAGCCTCCGCGTGATA CGAACCCTTTCGCCAATATATGACGACCACCGAAATAGTTCAGTCGACCCATCTACGCTCATGTGCGGAGACCAGCGCTGGTGACCTCTGAATTATGGAGACTCTAATCGCGACGACATCCCCAAGAATATGTCATGAAGGGCCAGGTCAATGCAA TCACTGCGGGCCACTTCAGTCGACAGCGTGTATATTGCCTAGATTCCTCTTGAAGCACTTGAGATAAATAACGGATCTTATCACCGGATATTTTGCTATCCACCTCATACCTTTAATTAAACCTTTTATCGGAACGCAGGGTGTACTCGGACTAGG TTCTGACTCTACGAGGGGGACCGCTTGGCTTTCCTTTGAGGCGGTTACATTGTCATTTTGTATTCAGTCTACCCGGGGGCCGACTAGTGGAGGCGGTGGCACATTCGGGCCATTGTCTACACAGTTGCTCGGGAAGCGGTGTTCAGATCCTGTTTA CTGGTTACACTCCGTTGTCGGTCCTAGGCTGTGATGTCTAGCCCCTGAATCAAAGCATGCCAGGGTTTCCAAGAGTCGGTACGTGTGCACCGTTGCCTTAAGTCTTAGTTCACGCCCTGGCTGAGAGTGTGCGCTATTGCAACTGGTTCATTCAAC CTAAGATTCCTTGGCCGTACATAAAGGCGCTGATCGACGCTGCCTTGGTGCGTACGGTGAGAAGCGTGCTTAATTTCAGATATCCCCTATCTGTCTGGATGTTATTCTCATTCATTCAACGTCGGTGGGCATCTCGTATGTTCAGATGGGATAGAC TCCACTCGTTCCATCAGCCGGTGGCCCTCGAGAGCTTTAGTGCTGACGAGAGGTGTCACAGGTTCAATCCACTCCTGTTTGGATTGTACAAGGGGCCAGGCTGGTTAGAAATGTTAACTCTCGCTACATCTGCGCGGGACGGCGCTAGTCAGTCGC CGGGAGGATCTTAGTCCCAACATTGATGCGGCTATAAGGAAGATAGGGCTGCCGTTTGGCACGTCTCGTTATTCAAAGCAAAATTTTGAGGCCATCCCAATAACCGTACCACAAGACAGAGTTTACTTCTCGGAGGGAAGATGCGCTTCATTCGAC TACGCGCCGAGCACCTGTGGAGTGGGTGTGGGCAAGACCCTAGTTACATGCGGCACGGGATTCTGGAAGTAAACATAGTATGCTCTGACACAACTTGATGACGATAACTATCCACAGAGCAAAATGATTCATCATCTTTTCGGGATTTCAGTCTAC CTCCACTAAGTACGCATATTCCAAACCTCAGACATCATACATGTATAAGTCTTATATGTTAGATAGGTTAAACCTTCAGAGTTTTTAGGCTAACTCTATAGGTTGAGCGGCATTGGACAGACACGTGGAGGCAATTCAGTCTACTCTAGGACCCGA TTGGGCATCCCGCTTCAGAGGGCCTACCCTCTTGGCACTGGGACGTTAAGTTCACTCTACTTAAATGCCCATTGGTGGGATCCTCCTTCTAGAATAACCGAACAAGCATTGTAAGTTATTCTTAGGAAAGGCACGTATGTCCAGTGAAGGGTCATC GAACCGGCCGCGCCTCTGGCAGAGTGCCGCTATCAGGCGCACCCCGCCATCCACATGTGGGATGGAGCTAATGAATGGCAACTAAGTTCACTCAACGTTGATTTCTCGAATGAATCCTGACCTGCTTGCCAGGGCGTGATTTATCATTTGGGTGAT TGGCTACATTAGTAACAGATCGAGGACTAACTGATGATTTCAGTCAACCTTGCCGGAAAGTGCGGGTGACATCCAAGAGTAGCAAGTGTGGACTCACGAGGCTTGAAGGATACTGCCTGGCGATCTTCATATATTGGCAGTCAACCTCAGTTCGTA AATCGAGTGGGGCTATTCCACAAATAAAACAACCATTAAATTGGCCCCGAAATCTAGACACAACCAATATAGAGTTCAGTCTACTACAGTTGGTCTGTTTGGCGTACCCGATCTCCGCCTCATTTAATCTTCGTTTCTAGAGATTTTCTCAATCGC".split()
k = 12
t = 25
best_motifs = greedy_motif_search(Dna, k, t)
for motif in best_motifs:
    print(motif)

TTCAGTCTACGG
GTTTACAGACGA
CATCTAACGTCC
TGGCCCCAACCT
TTCATTCTACGC
TTCACTCAACCA
TACCCACAGCGC
TTTATTCAATCC
TTCAATCAACGA
TTCAGTCTACCG
GTGCTTAGGCCC
TTCAATCCACGG
TTCAGTCGACCC
TGCTATCCACCT
TTCAGTCTACCC
TGCCTTAAGTCT
TTCATTCAACGT
TTCCATCAGCCG
CGGGAGGATCTT
CTGACACAACTT
TTCAGTCTACTC
TTCACTCTACTT
TTCACTCAACGT
TTCAGTCAACCT
TTCAGTCTACTA
